# Data Wrangling Notebook
Wrangles the data and creates 'wrang_xyz_data.csv'.

In [1]:
#Imports

#Data
import pandas as pd
import numpy as np

#Date
import datetime as dt
from datetime import datetime

In [5]:
#Function that wrangles the data
def wrangle_data(file='data/xyz_corp_lending_data.csv'):
    
    #Read CSV file
    df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')

    #Drop columns (to be explained why for each column)
    df=df.drop(['policy_code','funded_amnt','funded_amnt_inv','out_prncp_inv','total_pymnt_inv','id','member_id','total_rec_prncp','policy_code','installment','pymnt_plan','application_type','next_pymnt_d','title','emp_title'],axis=1)

    #Creating a metric to get % of null values and selecting the columns that have less than 50%
    nullity=(df.isnull().sum()/len(df)*100).sort_values(ascending=False)
    df=df[nullity[nullity<50].index]

    #Removing text from emp_length and term columns and converting to float
    df['emp_length']=df['emp_length'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
    df['term']=df['term'].str.replace('[A-Za-z\s+<>]+', '').astype('float')

    #Converting Date Columns to YYYYMMDD format and float
    date_cols=['last_pymnt_d','last_credit_pull_d','issue_d','earliest_cr_line']
    for col in date_cols:
        df[col]=pd.to_datetime(df[col]).dt.strftime('%Y%m%d').astype(float)

    #Filling na with median or mode
    med=['tot_cur_bal','tot_coll_amt','total_rev_hi_lim','revol_util']
    mod=['emp_length','last_pymnt_d','collections_12_mths_ex_med','last_credit_pull_d']
    for col in med:
        df[col]=df[col].fillna(df[col].median())

    for col in mod:
        df[col]=df[col].fillna(df[col].mode()[0])

    #Cleaning up zip_code to only have numbers
    df['zip_code']=df['zip_code'].str.replace('[A-Za-z\s+<>]+', '')#.astype('float')
    df['address']=df['zip_code']+df['addr_state']
    df=df.drop(['zip_code','addr_state'],axis=1)

    #Combning two rows that are the same in verification_status column
    df['verification_status']=df['verification_status'].str.replace('Source Verified','Verified')

    #Encoding categorical values 
    #cat_columns=['purpose','verification_status','home_ownership','initial_list_status','address','term']
    #df[cat_columns] = df[cat_columns].astype('category').apply(lambda x: x.cat.codes)

    #Cleaning outliers
    df=df[(df['dti']<150)&(df['revol_util']<200)&(df['acc_now_delinq']<13)&(df['tot_coll_amt']<1000000)&(df['tot_cur_bal']<4000000)&(df['total_rev_hi_lim']<2000000)&(df['out_prncp']<35001)]
    return df

#Running the function 
wrangle_data()#.to_csv('data/wrang_xyz_data.csv',index=False)

C:\Users\mmoy\AppData\Local\Temp\ipykernel_17820\931532063.py:5: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_17820\931532063.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['emp_length']=df['emp_length'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_17820\931532063.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['term']=df['term'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_17820\931532063.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['zip_code']=df['zip_code'].str.replace('[A-Za-z\s+<>]+', '')#.astype('float')


,tot_cur_bal,tot_coll_amt,total_rev_hi_lim,emp_length,last_pymnt_d,revol_util,collections_12_mths_ex_med,last_credit_pull_d,loan_amnt,acc_now_delinq,...,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_int,term,default_ind,address
0,81008.5,0.0,23800.0,10.0,20150101.0,83.7,0.0,20160101.0,5000.0,0.0,...,0.0,13648.0,9.0,f,0.00,5861.071414,861.07,36.0,0,860AZ
1,81008.5,0.0,23800.0,1.0,20130401.0,9.4,0.0,20130901.0,2500.0,0.0,...,0.0,1687.0,4.0,f,0.00,1008.710000,435.17,60.0,1,309GA
2,81008.5,0.0,23800.0,10.0,20140601.0,98.5,0.0,20160101.0,2400.0,0.0,...,0.0,2956.0,10.0,f,0.00,3003.653644,603.65,36.0,0,606IL
3,81008.5,0.0,23800.0,10.0,20150101.0,21.0,0.0,20150101.0,10000.0,0.0,...,0.0,5598.0,37.0,f,0.00,12226.302212,2209.33,36.0,0,917CA
4,81008.5,0.0,23800.0,1.0,20160101.0,53.9,0.0,20160101.0,3000.0,0.0,...,0.0,27783.0,38.0,f,766.90,3242.170000,1009.07,60.0,0,972OR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855964,25274.0,0.0,17100.0,8.0,20160101.0,82.1,0.0,20160101.0,10000.0,0.0,...,1.0,14037.0,15.0,f,7055.47,3971.880000,1027.35,36.0,0,905CA
855965,140285.0,0.0,10200.0,10.0,20160101.0,84.5,0.0,20160101.0,24000.0,0.0,...,0.0,8621.0,23.0,f,16933.26,9532.390000,2465.65,36.0,0,072NJ
855966,34178.0,0.0,18000.0,5.0,20160101.0,61.3,0.0,20160101.0,13000.0,0.0,...,1.0,11031.0,22.0,w,11155.50,3769.740000,1925.24,60.0,0,378TN
855967,58418.0,0.0,27000.0,1.0,20160101.0,30.6,1.0,20160101.0,12000.0,0.0,...,0.0,8254.0,20.0,w,10447.31,3787.670000,2234.98,60.0,0,010MA
